In [1]:
import requests
import pandas as pd

In [15]:
grouped_daily = requests.get('https://api.polygon.io/v2/aggs/grouped/locale/global/market/crypto/2022-02-17?adjusted=true&apiKey=81oGdC78et6jp5itDXcGQRqkK4FtPdmX')

In [17]:
gd_results = grouped_daily.json()['results']
pairs = [obj['T'].replace('X:', '') for obj in gd_results]
all_tickers = [x[0:-3] for x in pairs]
tickers = list(set(all_tickers))

In [4]:
def get_max_data(ticker, start_date, end_date):
    data = {}
    data[ticker] = {}
    try:
        res = requests.get('https://api.polygon.io/v2/aggs/ticker/X:{ticker}USD/range/1/minute/{start_date}/{end_date}?adjusted=true&sort=asc&limit=50000&apiKey=81oGdC78et6jp5itDXcGQRqkK4FtPdmX'
                           .format(ticker=ticker, start_date=start_date, end_date=end_date)).json()
        res_results = res['results']
        for key in res_results[0].keys():
            data[ticker][key] = [x[key] for x in res_results]
    except Exception as e:
        print(e)
        print(res)
    return data

In [5]:
def get_all_max(ticker_list, start_date, end_date):
    lst = []
    for ticker in ticker_list:
        res = get_max_data(ticker, start_date, end_date)
        lst.append(res)
    return lst

In [323]:
tickers.sort()

In [324]:
tickers_slice = tickers[:5]

In [326]:
top_five = ['BTC', 'ETH', '1INCH', 'AAVE', 'MATIC']

In [327]:
top_lst = get_all_max(top_five)

In [6]:
def build_dfs(results):
    v, vw, o, c, l, h, t = {}, {}, {}, {}, {}, {}, []
    index = []
    for count, obj in enumerate(results):
        ticker = list(obj.keys())[0]
        for x in obj.values():
            for key, vals in x.items():
                if len(vals) < 50000:
                    diff = 50000 - len(vals)
                    vals.extend([None for x in range(diff)])
                if key == 'v':
                    v[ticker] = vals
                elif key == 'vw':
                    vw[ticker] = vals
                elif key == 'o':
                    o[ticker] = vals
                elif key == 'c':
                    c[ticker] = vals
                elif key == 'l':
                    l[ticker] = vals
                elif key == 'h':
                    h[ticker] = vals
                elif key == 't' and len(t) == 0 and len(vals) >= 50000:
                    t = pd.Series(vals)*1000000

    index = [x for x in range(50000)]
    dt_index = pd.DatetimeIndex(t)
    multi_dex = pd.MultiIndex.from_tuples(zip(index, dt_index), names=['index', 'datetime'])

    vdf = pd.DataFrame(v, index=multi_dex).loc[:48960]
    vwdf = pd.DataFrame(vw, index=multi_dex).loc[:48960]
    odf = pd.DataFrame(o, index=multi_dex).loc[:48960]
    cdf = pd.DataFrame(c, index=multi_dex).loc[:48960]
    hdf = pd.DataFrame(h, index=multi_dex).loc[:48960]
    ldf = pd.DataFrame(l, index=multi_dex).loc[:48960]

    dfs = {'v': vdf, 'vw': vwdf, 'o': odf, 'c': cdf, 'h': hdf, 'l': ldf}
    return dfs

In [374]:
dfs = build_dfs(top_lst)

In [379]:
o = dfs['o']

In [384]:
o.isna().sum()

BTC         0
ETH         0
1INCH    5242
AAVE        0
MATIC       0
dtype: int64

In [391]:
o.loc[len(o)-5242:, '1INCH']

index  datetime           
43719  2021-12-22 08:39:00   NaN
43720  2021-12-22 08:40:00   NaN
43721  2021-12-22 08:41:00   NaN
43722  2021-12-22 08:42:00   NaN
43723  2021-12-22 08:43:00   NaN
                              ..
48956  2021-12-26 23:56:00   NaN
48957  2021-12-26 23:57:00   NaN
48958  2021-12-26 23:58:00   NaN
48959  2021-12-26 23:59:00   NaN
48960  2021-12-27 00:00:00   NaN
Name: 1INCH, Length: 5242, dtype: float64

In [32]:
inch_dec = get_all_max(['1INCH'], '2021-12-22', '2022-02-05')

In [33]:
inch_dfs = build_dfs(inch_dec)

In [34]:
inch_o = inch_dfs['o']

In [35]:
inch_o.isna().sum()

1INCH    1155
dtype: int64

In [36]:
inch_o.dropna()

,,1INCH
index,datetime,
0,2021-12-22 00:00:00,2.44
1,2021-12-22 00:02:00,2.44
2,2021-12-22 00:03:00,2.44
3,2021-12-22 00:04:00,2.44
4,2021-12-22 00:06:00,2.45
...,...,...
47801,2022-02-05 23:47:00,1.79
47802,2022-02-05 23:51:00,1.79
47803,2022-02-05 23:55:00,1.79
